In [62]:
from Models.model_sources.markov_source import MarkovChain
from Models.MMC import MMC
from Models.HMC import HMC
from Models.DBN import FMC
from Models.model_sources.mtd_source import MTD

from Datasets import Blocksworld_Data, Markov_Data_Casual

from Datasets.Markov_Data import HMM_Data
from Datasets.MMC_Data import MMC_data
from Datasets.Markov_Data_Large import HMM_Decisive
from Datasets.Fruit_Data import fruit_domain
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

import warnings

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import pandas as pd
import numpy as np
import seaborn as sns

from mtdlearn.mtd import MTD, RandomWalk
from mtdlearn.preprocessing import PathEncoder, SequenceCutter

In [64]:
import requests
import pandas as pd

def get_crypto_price(symbol, exchange, start_date = None):
    api_key = 'QEUZDNSCRV22B7VL'
    api_url = f'https://www.alphavantage.co/query?function=CRYPTO_INTRADAY&symbol={symbol}&market={exchange}&apikey={api_key}&interval=30min&outputsize=full&page=1'
    raw_df = requests.get(api_url).json()
    df = pd.DataFrame(raw_df['Time Series Crypto (30min)']).T
    #df = df.rename(columns = {'1a. open (USD)': 'open', '2a. high (USD)': 'high', '3a. low (USD)': 'low', '4a. close (USD)': 'close', '5. volume': 'volume'})
    for i in df.columns:
        df[i] = df[i].astype(float)
    df.index = pd.to_datetime(df.index)
    #df = df.iloc[::-1].drop(['1b. open (USD)', '2b. high (USD)', '3b. low (USD)', '4b. close (USD)', '6. market cap (USD)'], axis = 1)
    if start_date:
        df = df[df.index >= start_date]
    return df

df = get_crypto_price(symbol = 'BTC', exchange = 'USD')
df['Change'] = df['4. close'].diff()

df['Change_enc'] = np.nan

df.loc[df.Change < 0.0, 'Change_enc'] = '1_DROP'
#df.loc[df.Change < -40, 'Change_enc'] = '0_HUGE_DROP'

df.loc[df.Change > 0.0, 'Change_enc'] = '2_RISE'
#df.loc[df.Change > 40, 'Change_enc'] = 'HUGE_RISE'

#df.loc[df.Change.between(-5, 5), 'Change_enc'] = 'NO_CHANGE'

df.dropna(inplace=True)


In [65]:
import requests
import pandas as pd

def get_crypto_price(symbol, exchange, start_date = None):
    api_key = 'QEUZDNSCRV22B7VL'
    api_url = f'https://eodhistoricaldata.com/api/intraday/BTC-USD.CC?api_token=630d712fd8bd74.01254339&interval=5m&fmt=json'
    raw_df = requests.get(api_url).json()
    df = pd.DataFrame(raw_df)
    #df = df.rename(columns = {'1a. open (USD)': 'open', '2a. high (USD)': 'high', '3a. low (USD)': 'low', '4a. close (USD)': 'close', '5. volume': 'volume'})
    df['close'] = df['close'].astype(float)
    #df.index = pd.to_datetime(df.index)
    #df = df.iloc[::-1].drop(['1b. open (USD)', '2b. high (USD)', '3b. low (USD)', '4b. close (USD)', '6. market cap (USD)'], axis = 1)
    return df

df = get_crypto_price(symbol = 'BTC', exchange = 'USD')
df['Change'] = df['close'].diff()

df['Change_enc'] = np.nan
std = df.std()['Change']

df.loc[df.Change < 0.0, 'Change_enc'] = 'drop'
df.loc[df.Change <= -std, 'Change_enc'] = 'big_drop'
df.loc[df.Change <= -2*std, 'Change_enc'] = 'bigger_drop'
df.loc[df.Change <= -3*std, 'Change_enc'] = 'biggest_drop'

df.loc[df.Change > 0.0, 'Change_enc'] = 'rise'
df.loc[df.Change >= std, 'Change_enc'] = 'big_rise'
df.loc[df.Change >= 2*std, 'Change_enc'] = 'bigger_rise'
df.loc[df.Change >= 3*std, 'Change_enc'] = 'biggest_rise'

#df.loc[df.Change.between(-5, 5), 'Change_enc'] = 'NO_CHANGE'

df.dropna(inplace=True)

/tmp/ipykernel_8584/2598880382.py:19: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  std = df.std()['Change']


In [45]:
df

,timestamp,gmtoffset,datetime,open,high,low,close,volume,Change,Change_enc
2,1651536600,0,2022-05-03 00:10:00,38543.230468,38553.597656,38543.230468,38553.597656,27320320.0,41.195313,rise
11,1651539300,0,2022-05-03 00:55:00,38452.425781,38452.425781,38432.628906,38432.628906,22528.0,-22.191406,drop
16,1651540800,0,2022-05-03 01:20:00,38391.320312,38401.191406,38391.320312,38401.191406,12023808.0,0.035156,rise
17,1651541100,0,2022-05-03 01:25:00,38403.914062,38408.726562,38403.914062,38407.105468,10377216.0,5.914062,rise
19,1651541700,0,2022-05-03 01:35:00,38437.101562,38469.343750,38437.101562,38469.343750,56254464.0,63.640625,big_rise
...,...,...,...,...,...,...,...,...,...,...
34405,1661824800,0,2022-08-30 02:00:00,20168.687500,20169.841796,20165.941406,20166.732421,524288.0,-1.300782,drop
34406,1661825100,0,2022-08-30 02:05:00,20165.722656,20165.876953,20162.320312,20162.320312,15710208.0,-4.412109,drop
34407,1661825400,0,2022-08-30 02:10:00,20159.646484,20159.646484,20152.353515,20152.353515,4253696.0,-9.966797,drop
34408,1661825700,0,2022-08-30 02:15:00,20152.031250,20152.031250,20147.273437,20151.773437,9611264.0,-0.580078,drop


df = pd.read_csv("/mnt/watchandhelp/PycharmProjects/mtd-learn/examples/euro_usd.csv")

df['Change'] = df.Closing_rate.diff()

df['Change_enc'] = np.nan

df.loc[df.Change < 0.0, 'Change_enc'] = '1_DROP'
df.loc[df.Change < -0.005, 'Change_enc'] = '0_BIG_DROP'
df.loc[df.Change >= 0, 'Change_enc'] = '2_RISE'
df.loc[df.Change >= 0.005, 'Change_enc'] = '3_BIG_RISE'

df.dropna(inplace=True)

## Fit models

In [56]:
aics = []
bics = []

In [77]:
from sklearn.model_selection import train_test_split

order = 4

sc = SequenceCutter(order)
x, y = sc.transform(df.Change_enc.values)

pe = PathEncoder(order)
pe.fit(x, y)

x_tr, y_tr = pe.transform(x, y)


In [78]:
pe.label_dict_inverse

{0: 'big_drop',
 1: 'big_rise',
 2: 'bigger_drop',
 3: 'bigger_rise',
 4: 'biggest_drop',
 5: 'biggest_rise',
 6: 'drop',
 7: 'rise',
 8: 'null'}

In [79]:

div = round(len(x_tr)*.85)
#(X_train, X_test, y_train, y_test) = train_test_split(x_tr, y_tr)
X_train, X_test, y_train, y_test = x_tr[div:, :],x_tr[:div, :], y_tr[div:], y_tr[:div]
#X_train, y_train = x_tr, y_tr
#X_test, y_test = x_tr, y_tr
amount_to_average = 1

training_master = []
testing_master = []

sgo_type = "greedy"
methods = [HMC, MMC, FMC, MTD] #FMC]
types = [m.__name__ for m in methods]

state_count = len(np.unique(x_tr))
for _ in range(amount_to_average):

    args_training = {"X_train": X_train, "y_train": y_train}
    args_testing =   {"X_test": X_test, "y_test": y_test}
    results_training = []
    results_testing = []

    for m in methods:
        model = m(state_count, order=order)
        training = MarkovChain.calculate_time(model.train, args_training)
        testing = MarkovChain.calculate_time(model.test, args_testing)


        pred_res = model.predict(X_test)
        count = 0
        for i,p in enumerate(pred_res):
            if "drop" in pe.label_dict_inverse[p] and "drop" in pe.label_dict_inverse[y_test[i]]:
                count += 1
            elif "rise" in pe.label_dict_inverse[p] and "rise" in pe.label_dict_inverse[y_test[i]]:
                count += 1
        print(count/len(pred_res))


        #print(count/sum)

        print(model.__class__.__name__)
        print(f"Training: {training}")
        print(f"Testing: {testing}")
        print("")

    print(results_training)
    print(results_testing)
    training_master.append(results_training)
    testing_master.append(results_testing)

0.5233526085022558
HMC
Training: (None, 0.103427)
Testing: (0.36588930545847365, 2.375991)

0.5754591091059287
MMC
Training: (None, 0.039448)
Testing: (0.44309588867001337, 0.525861)

0.5609074156446591
FMC
Training: (None, 0.070985)
Testing: (0.42047404206646755, 2.862221)

0.5666264218084769
MTD
Training: (None, 2.392575)
Testing: (0.43019635254495775, 0.002582)

[]
[]


In [ ]:
X_train

## Choose model

In [22]:
xs = [0, 1, 2, 3, 4]

In [18]:
sns.lineplot(x=xs, y=aics)
sns.lineplot(x=xs, y=bics);

ValueError: All arrays must be of the same length